In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def cleanData(df,substance):
    df = df.drop(columns=['historianTagnummer'])
    df.rename(columns={'hstWaarde': substance + 'Value'}, inplace=True)
    df[substance + 'Value'] = df[substance + 'Value'].astype(float)
    df = df.drop(columns=['datumBeginMeting'])
    df['datumEindeMeting'] = pd.to_datetime(df['datumEindeMeting'])
    df.rename(columns={'datumEindeMeting': 'measurementDate'}, inplace=True)
    df.to_parquet('../../data/cleanedData/'+ substance +'.parquet', index=False)
    pass

In [ ]:
data_files = [
    ('../../data/Ammonium_measurements.parquet', 'ammonium'),
    ('../../data/Nitrate_measurements.parquet', 'nitrate'),
    ('../../data/Oxygen_A.parquet', 'oxygenA'),
    ('../../data/Oxygen_B.parquet', 'oxygenB'),
    ('../../data/Phosphate_measurements.parquet', 'phosphate')
]

for file_path, var_name in data_files:
    df = pd.read_parquet(file_path)
    cleanData(df, var_name)

In [ ]:
total = pd.read_csv('../../data/Total_influent_flow_WWTP_Ede_2021_minute_data.csv', sep=';')
total.drop(columns=['wwResolution'],inplace=True)
total['DateTime'] = pd.to_datetime(total['DateTime'])
total.rename(columns={'EDE_09902MTW_K100.MTW':'waterFlowPerMinute','DateTime':'measurementDate'}, inplace=True)

total['waterFlowPerMinute'] = total['waterFlowPerMinute'].str.replace(',','.').replace('(null)', np.nan)
total['waterFlowPerMinute'] = total['waterFlowPerMinute'].astype('float')

total = total.sort_values(by='measurementDate')
total.to_parquet('../../data/shiftedDates/total.parquet',index=False)

In [ ]:
print(total.info())
print(total.describe())

In [ ]:
phosphate = pd.read_parquet('../../data/cleanedData/phosphate.parquet')

start_date = '2021-01-01 00:00:00'
end_date = '2021-12-31 23:58:00'
index = pd.date_range(start=start_date, end=end_date, freq='1min')
phosphate = phosphate.set_index('measurementDate').reindex(index, fill_value=None)
phosphate = phosphate.reset_index()
phosphate.rename(columns={'index': 'measurementDate'}, inplace=True)
phosphate.to_parquet('../../data/shiftedDates/phosphate.parquet')